In [1]:
import findspark
findspark.init()
findspark.find()
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

In [2]:
from datetime import date, timedelta, datetime
import pyspark.sql.functions as F
from pyspark.sql.window import Window 
from pyspark.sql import DataFrame

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.master("yarn")\
.config("spark.driver.memory", "2g") \
.config("spark.executor.memory", "2g") \
.appName("Project8") \
.getOrCreate()                               

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2023-01-26 18:30:30,086 WARN util.Utils: Your hostname, fhmst38qln6c3o1fij70 resolves to a loopback address: 127.0.1.1; using 172.16.0.19 instead (on interface eth0)
2023-01-26 18:30:30,087 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-01-26 18:30:32,338 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update 

# Функции

In [4]:
# Выбирает все даты, попадающие в заданный диапазон.
# Даты передаются в формате "2022-01-01"
def select_dates(start_date: str, end_date: str) -> list:
    try:
        start_date = datetime.fromisoformat(start_date)
        end_date = datetime.fromisoformat(end_date)
    except:
        print('Неверный формат даты!') # ПИСАТЬ В ЛОГ!!!!!
    # Список дат в формате ['/user/master/data/events/date=2020-10-02', ...]
    paths_list = []
    # Количество дней между датами
    delta = end_date - start_date
    
    if delta.days < 0:
        print ('Указан слишком маленький диапазон!')  # ПИСАТЬ В ЛОГ!!!!!
    for i in range(delta.days + 1):
        paths_list.append('/user/master/data/geo/events/date=' + (start_date + timedelta(i)).__str__()[:10])
    return paths_list

calculate_dist = F.lit(2*6371) * F.asin(
        F.sqrt(
            F.pow(F.sin((F.radians(F.col("lat")) - F.radians(F.col("ltt"))) / 2), 2) +
            F.cos(F.radians(F.col("lat"))) * F.radians(F.cos(F.col("ltt"))) *
            F.pow(F.sin((F.radians(F.col("lon")) - F.radians(F.col("lng"))) / 2), 2)
        )
    )


In [5]:
select_dates('2022-01-01', '2022-01-03')

['/user/master/data/geo/events/date=2022-01-01',
 '/user/master/data/geo/events/date=2022-01-02',
 '/user/master/data/geo/events/date=2022-01-03']

## Загрузка данных

### Действия

In [5]:
#  Добавить проверку на количество записей в лог
try:
    activities = spark.read.parquet(*select_dates('2022-01-01', '2022-01-02'))
except:
    print('Нет данных за указанный диапазон!')

In [6]:
activities.show(3, truncate=False)

+--------------------------------------------------------------------------+------------+------------------+------------------+
|event                                                                     |event_type  |lat               |lon               |
+--------------------------------------------------------------------------+------------+------------------+------------------+
|[,, 2022-01-01 19:39:54,,,,,,,,,,,, 376484,, 141560]                      |subscription|-34.79058202339126|149.62371004159567|
|[,,,, same OS,, 9040,, 593944, 62475, 2021-01-01 13:51:56.362816053,,,,,,]|message     |-32.27676278030154|152.01807950723008|
|[,, 2022-01-01 01:45:29,,,,,,,,,,,, 387849,, 251849]                      |subscription|-42.22344734393438|147.75004888501377|
+--------------------------------------------------------------------------+------------+------------------+------------------+
only showing top 3 rows



In [8]:
activities.schema

StructType(List(StructField(event,StructType(List(StructField(admins,ArrayType(LongType,true),true),StructField(channel_id,LongType,true),StructField(datetime,StringType,true),StructField(media,StructType(List(StructField(media_type,StringType,true),StructField(src,StringType,true))),true),StructField(message,StringType,true),StructField(message_channel_to,LongType,true),StructField(message_from,LongType,true),StructField(message_group,LongType,true),StructField(message_id,LongType,true),StructField(message_to,LongType,true),StructField(message_ts,StringType,true),StructField(reaction_from,StringType,true),StructField(reaction_type,StringType,true),StructField(subscription_channel,LongType,true),StructField(subscription_user,StringType,true),StructField(tags,ArrayType(StringType,true),true),StructField(user,StringType,true))),true),StructField(event_type,StringType,true),StructField(lat,DoubleType,true),StructField(lon,DoubleType,true)))

### Города

In [7]:
cities = spark.read.csv('/user/flaviusoct/data/coord', sep=';', header=True).withColumnRenamed('lat', 'ltt')

In [8]:
cities.show(3)

+---+---------+--------+--------+
| id|     city|     ltt|     lng|
+---+---------+--------+--------+
|  1|   Sydney| -33,865|151,2094|
|  2|Melbourne|-37,8136|144,9631|
|  3| Brisbane|-27,4678|153,0281|
+---+---------+--------+--------+
only showing top 3 rows



## Обработка данных

In [9]:
# Удаляем строки с пропущенными значениями координат

activities_new = activities.where("lat IS NOT NULL AND lon IS NOT NULL")

In [10]:
# Делаем одну колонку со временем
# и добавляем колонку с уникальным id
activities_new = activities_new \
.withColumn('datetime', F.coalesce(F.col('event.datetime'), F.col('event.message_ts')).cast('timestamp')) \
.orderBy(F.asc('datetime')) \
.withColumn('activity_id', F.monotonically_increasing_id())
# В этом точно есть, ибо группировать ниже надо по какому-то общему идентификатору
# в сообщениях и иных действиях. По умолчанию его нет.


In [ ]:
activities_new.show(3, truncate=False)

In [ ]:
cities_new = cities \
.withColumn("ltt", F.regexp_replace(F.col("ltt"), pattern=',', replacement='.').cast("double")) \
.withColumn("lng", F.regexp_replace(F.col("lng"), pattern=',', replacement='.').cast("double"))

## Операции

In [ ]:
activities_new = activities_new.crossJoin(cities_new) \
        .withColumn('distance', calculate_dist) \
        .withColumn("distance_rank",
                    F.row_number().over(Window.partitionBy("activity_id").orderBy("distance"))) \
        .where("distance_rank == 1")

In [ ]:
activities_new.show()

## Создаём витрину

In [22]:
def create_user_view(df: DataFrame) -> DataFrame:
    messages = df.where("event_type='message' AND event.message_ts IS NOT NULL") \
    .select(F.col('event.message_from').alias('user_id'),
            F.to_date(F.col('event.message_ts')).alias('msg_date'),
            F.col('city')
           ).distinct()
    
    '''
    act_city = messages \
    .withColumn(
        'user_messages_rank', F.row_number().over(Window().partitionBy(['user_id']).orderBy(F.desc('msg_date')))
    ).where('user_messages_rank == 1') \
    .select(F.col('user_id'), F.col('city').alias('act_city'))
    
    home_city = messages. \
    withColumn("date_group_rank",
               F.row_number().over(Window.partitionBy("user_id", "city").orderBy("msg_date"))) \
    .selectExpr('*', 'date_sub(msg_date, date_group_rank) as date_group') \
    .groupBy("user_id", "city", "date_group") \
    .agg(F.count("*").alias("cnt_days")) \
    .select("user_id", "city", "cnt_days") \
    .where("cnt_days > 1") \
    .withColumn('max_cnt', F.max('cnt_days').over(Window.partitionBy("user_id", "city"))) \
    .where(F.col('cnt_days') == F.col('max_cnt')) \
    .drop('max_cnt', 'cnt_days') \
    .withColumn("home_city", F.first("city").over(Window.partitionBy("user_id"))) \
    .drop("city") \
    #.withColumnRenamed("city", "home_city")

    '''
    travel_count = messages \
    .groupBy("user_id") \
    .agg(F.count("*").alias("travel_count"))
    
    travel_array = messages \
    .orderBy("msg_date") \
    .groupBy("user_id") \
    .agg(F.collect_list("city").alias("travel_array")) \
    .drop("city", "msg_date")
    
    print(1)
    local_time = messages \
    .withColumn('message_rank_by_user',
                F.row_number().over(Window.partitionBy("user_id").orderBy(F.desc('msg_date')))
               ).where('message_rank_by_user == 1') \
    .select(F.col('user_id'), F.col('msg_date').alias('')).orderBy(F.col('user_id'))
    
    print(2)
    user_registration = messages \
        .withColumn('message_order_rank', F.min('msg_date').over(Window.partitionBy("user_id"))) \
        .where(F.col('msg_date') == F.col('message_order_rank')) \
        .select(F.col("user_id"), F.col('msg_date')) \
        .groupBy("user_id") \
        .agg(F.first("msg_date")).orderBy(F.col('user_id'))
    
    local_time.show(7)
    user_registration.show(7)

In [31]:
activities_new.show(truncate=False)

KeyboardInterrupt: 

In [30]:
spark._jsc.c

In [ ]:
!hdfs dfs -ls /user/master/data/geo/events/date=2022-01-01